# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Требуется обучить модель и классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.


## Подготовка к работе

Установим все требуемые для выполнения проектной работы фреймворки и библиотеки:

In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install detoxify

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from catboost import CatBoostClassifier
import torch
import transformers
from tqdm import notebook
from transformers import AutoModel
from detoxify import Detoxify
from sklearn.pipeline import Pipeline


nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alex/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
#Загрузим данные
df = pd.read_csv('toxic_comments.csv')


In [ ]:
#Ознакомимся
print(df.head())
print(df.shape)

## Предобработка

In [ ]:
#Извлечем корпус
corpus = df['text'].values

In [ ]:
#Напишем функцию очистки текста при помощи регулярных выражений
def clean_text(text):
    txt_after_sub = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(txt_after_sub.split())


In [ ]:
#Напишем функцию для получения POS-тегов для лемматизатора WordNet
nltk.download('wordnet')

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J":wordnet.ADJ,
                "V":wordnet.VERB,
                "R":wordnet.ADV,
                "N":wordnet.NOUN}
    return tag_dict.get(tag,wordnet.NOUN)


In [ ]:
#Собственно сам лемматизатор с разбивкой текста на токены внутри
Lemmatizer = WordNetLemmatizer()

def lemmatizer(text):
    tokenized_text = nltk.word_tokenize(text)
    lemm_text = ' '.join([Lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokenized_text])
    return lemm_text

In [ ]:
#Создадим новый столбец в исходном датасете с лемматизированным и очищенным текстом
df['lemm'] = pd.Series(corpus).apply(lambda x: lemmatizer(clean_text(x)))
df.head()

In [ ]:
#Скачаем датасет для того чтобы не проделывать заново предобработку в случае если ядро отвалится
df.to_csv('df_lemm.csv')

In [5]:
#Загрузка датасета на случай если всё пошло не так:)
df = pd.read_csv('df_lemm.csv')

In [6]:
df.isna().sum()

Unnamed: 0    0
text          0
toxic         0
lemm          7
dtype: int64

In [7]:
df = df.dropna()

In [8]:
df.isna().sum()

Unnamed: 0    0
text          0
toxic         0
lemm          0
dtype: int64

In [9]:
#Определим фичи и таргет, а затем поделим их на тестовые и обучающие
features = df['lemm'].values
target = df['toxic'].values
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.25, random_state=12)



In [ ]:
features_train

In [10]:
#Посчитаем Tfidf с указанными стоп словами, затем обучим векторайзер и преобразуем им фичи
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))
count_tfidf = TfidfVectorizer(stop_words=stopwords)


tfidf_train = count_tfidf.fit_transform(features_train)
tfidf_test = count_tfidf.fit(features_train, target_train)
tfidf_test = count_tfidf.transform(features_test)



[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Подбор гиперпараметров и обучение моделей

### LogisticRegression

In [11]:
#Подберем гиперпараметры для логистической регрессии при помощи RandomizedSearch'а
#поскольку метод имеет внутри кросс-валидацию, выделять данные на валидацию не будем
model_LR = LogisticRegression()
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', LogisticRegression())
                    ])
params = {'tfidf__ngram_range':((1,1),(1,2)),
          'clf__C':range(1,5),
         'clf__max_iter':range(1,1000,20),
         'clf__random_state':[12]}
rand_search = RandomizedSearchCV(pipeline, param_distributions=params, scoring='f1', n_jobs=-1, verbose=1)
rand_search.fit(features_train, target_train)
print(f'Best score: {rand_search.best_score_}')
print(f'Best params: {rand_search.best_params_}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/alex/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/alex/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

Best score: 0.7622499134216033
Best params: {'tfidf__ngram_range': (1, 1), 'clf__random_state': 12, 'clf__max_iter': 641, 'clf__C': 4}


In [14]:
#а теперь то же самое на тестовой выборке
model_LR = LogisticRegression(random_state=12, max_iter=641, C=4, solver='liblinear', penalty='l1')
model_LR.fit(tfidf_train, target_train)
predictions = model_LR.predict(tfidf_test)


f1_score(target_test, predictions)

0.7809990472301619

**Отлично, сразу получилась проходная метрика, теперь попробуем другие алгоритмы.**

### CatBoostClassifier

In [15]:
#Посмотрим как себя покажет наш любимый бустинг
model_Cat = CatBoostClassifier(iterations=1000,
                              learning_rate=0.15)
model_Cat.fit(tfidf_train, target_train)
predictions_Cat = model_Cat.predict(tfidf_test)


f1_score(target_test, predictions_Cat)


0:	learn: 0.5479700	total: 504ms	remaining: 8m 23s
1:	learn: 0.4478161	total: 860ms	remaining: 7m 8s
2:	learn: 0.3810655	total: 1.2s	remaining: 6m 39s
3:	learn: 0.3342243	total: 1.55s	remaining: 6m 25s
4:	learn: 0.3023466	total: 1.9s	remaining: 6m 18s
5:	learn: 0.2813762	total: 2.26s	remaining: 6m 15s
6:	learn: 0.2663177	total: 2.63s	remaining: 6m 13s
7:	learn: 0.2548055	total: 3.01s	remaining: 6m 13s
8:	learn: 0.2461740	total: 3.36s	remaining: 6m 9s
9:	learn: 0.2392000	total: 3.7s	remaining: 6m 6s
10:	learn: 0.2340310	total: 4.04s	remaining: 6m 3s
11:	learn: 0.2298680	total: 4.39s	remaining: 6m 1s
12:	learn: 0.2262336	total: 4.73s	remaining: 5m 59s
13:	learn: 0.2232626	total: 5.07s	remaining: 5m 57s
14:	learn: 0.2195223	total: 5.43s	remaining: 5m 56s
15:	learn: 0.2173005	total: 5.77s	remaining: 5m 54s
16:	learn: 0.2152071	total: 6.12s	remaining: 5m 53s
17:	learn: 0.2127861	total: 6.48s	remaining: 5m 53s
18:	learn: 0.2112233	total: 6.82s	remaining: 5m 51s
19:	learn: 0.2088538	total: 7.

0.7576239340945223

**Катбуст дал f1 - 0.75, вроде метрика проходная, но посмотрим что покажет bert**

### Bert 

In [16]:
#Посмотрим на балланс классов
rat = pd.Series(target).value_counts()[1]/pd.Series(target).value_counts()[0]
rat

0.1131931993386308

Дисбаланс классов налицо, будем устранять.

In [17]:
#Напишем функцию по устраниению дисбалланса классов, в нашем случае целесообразнее выполнить downsampling так как Bert модель тяжелая
#и много объектов ядро домашней машины просто непотянет и отвалится
def downsampling(features, target, fraction):
    features_zeros=features[target==0]
    features_ones=features[target==1]
    target_ones=target[target==1]
    target_zeros=target[target==0]
    
    features_downsampled = pd.concat(
    [features_zeros.sample(frac=fraction, random_state=12)] + [features_ones])
    
    target_downsampled = pd.concat(
    [target_zeros.sample(frac=fraction, random_state=12)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=12)
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsampling(pd.Series(features_train), pd.Series(target_train), rat)
print(features_downsampled.shape, target_downsampled.shape)

(24353,) (24353,)


Готово, вот теперь другое дело.

In [18]:
#Возьмем небольшую подвыборку наших данных для модели Bert, таргет берется меньше уже с учётом последующей фильтрации фич по размеру токена
features_downsampled = features_downsampled.sample(800)
target_downsampled = target_downsampled.sample(800)

In [19]:
#Токенизируем данные и проверим размер получившихся векторов
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')


tokenized = features_downsampled.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

max_len


512

In [20]:
#Приведем все вектора к одному размеру путем добавления нулей 
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
padded.shape

(800, 512)

In [21]:
#Создадим маску внимания, чтобы модель обращала внимание на ненулевые токены в векторе 
attention_mask = np.where(padded !=0, 1, 0)
attention_mask.shape

(800, 512)

In [22]:
#Загрузим предобученную модель и преобразуем ей наш датасет в векторные представления - embedding'и
model = AutoModel.from_pretrained("distilbert-base-uncased")


batch_size = 10
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    embeddings.append(batch_embeddings[0][:,0,:].numpy())
embeddings

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/80 [00:00<?, ?it/s]

[array([[-0.09659109,  0.006181  , -0.00133071, ..., -0.10542802,
          0.3846906 ,  0.4091116 ],
        [ 0.32216868,  0.2822963 , -0.23127265, ..., -0.26801816,
          0.5261541 ,  0.05518529],
        [ 0.07144033,  0.04216814,  0.00959358, ..., -0.16574593,
         -0.03998029,  0.2828487 ],
        ...,
        [ 0.07031427,  0.05913085,  0.1047712 , ..., -0.10739137,
          0.5102302 ,  0.4495869 ],
        [-0.16052523,  0.02816637, -0.20893736, ..., -0.21196859,
          0.25574112,  0.41466814],
        [-0.04135018, -0.05734237,  0.0627276 , ..., -0.08693473,
          0.38504317,  0.3606385 ]], dtype=float32),
 array([[-0.38522106, -0.1355881 , -0.13347562, ..., -0.13157031,
          0.53072244,  0.4376408 ],
        [ 0.09495325,  0.28211328, -0.07077929, ...,  0.04767526,
          0.30706087,  0.23943298],
        [ 0.23659882, -0.00561754,  0.08020873, ..., -0.33377242,
          0.40538228,  0.15144567],
        ...,
        [ 0.21688077,  0.1175843 , -0.0

In [23]:
#Получилась солжная таблица состоящая из списка матриц, присоединим все эти матрицы друг к другу при помощи конкатенации и поделим их
#на тестовую с обучающей выборки
bert_features = np.concatenate(embeddings)
features_train_bert, features_test_bert, target_train_bert, target_test_bert = train_test_split(bert_features,
                                                                                               target_downsampled,
                                                                                               test_size=.25,
                                                                                               random_state=12)


In [24]:
#Обучим логистическую регрессию на полученной натрицу, сделаем предсказание по обучающих фичах и снимем метрику
model_LR_bert = LogisticRegression(random_state=12, max_iter=521, C=4, solver='liblinear', penalty='l1')
model_LR_bert.fit(features_train_bert, target_train_bert)
predictions_bert = model_LR_bert.predict(features_train_bert)


f1_score(target_train_bert, predictions_bert)

0.948073701842546

In [25]:
#Обучим логистическую регрессию на полученной натрицу, сделаем предсказание по тестовых фичах и снимем метрику
model_LR_bert = LogisticRegression(random_state=12, max_iter=521, C=4, solver='liblinear', penalty='l1')
model_LR_bert.fit(features_train_bert, target_train_bert)
predictions_bert = model_LR_bert.predict(features_test_bert)


f1_score(target_test_bert, predictions_bert)

0.45901639344262296

## Вывод:
Лучше всех себя показала логистическая регрессия с гиперпараметрами подобранными RandomizedSearch'ем.
Снять хорошую метрику с Bert не получилось в силу ограниченности ресурсов, при подаче больше тысячи объектов для обработки ядро в юпитере умирало. Пришлось очень сильно урезать выборку, чтобы домашняя машина смогла переварить этого Microsoft-овского зверя.